<a href="https://colab.research.google.com/github/neemiasbsilva/MLLMs-Teoria-e-Pratica/blob/main/use-cases/Classify_Sentiment_DeepseekVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepSeek VL - Classificação de Sentimento

Bem vindos a esse notebook com um pequeno tutorial para inferência em uma LLM multimodal, com um pedacinho de interface com pandas!

É muito importante que cada célula seja executada em sequência, conforme será explicado em logo mais.

Neste tutorial, devido às limitações do google Collab, vamos utilizar o [Deepseek-VL](https://github.com/deepseek-ai/DeepSeek-VL). Então, Vamos começar clonando seu repositório:

## Configuração do Ambiente

In [ ]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL -q

fatal: destination path 'DeepSeek-VL' already exists and is not an empty directory.


Em seguida, vamos instalar as dependências, tomando cuidado para instalar o Torch separado, já que este é sensível a versão que estamos utilizando de Python:

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q
!sed -i '/torch==/d' /content/DeepSeek-VL/requirements.txt
!pip install -r /content/DeepSeek-VL/requirements.txt -q

Após a instalação precisa reinicar a execução para que os pacotes necessários sejam devidamente instalados no kernel

## Carregar o Modelo e Inicialização do Tokenizer

Agora, vamos utilizar os métodos da biblioteca transformers e o torch para carregar o modelo na nossa placa gráfica, inicializar nosso tokenizer, que será responsável pela codificação das entradas, e decodificação das saídas para o modelo:

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append('/content/DeepSeek-VL')

import torch
from transformers import AutoModelForCausalLM

from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM

model_path = "deepseek-ai/deepseek-vl-1.3b-base"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

Python version is above 3.10, patching the collections module.


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

## Carregando o Modelo e Fazendo um Teste de Infêrencia

Vamos rodar este teste para verificar se o modelo está carregado corretamente! Note a reposta sendo mostrada no terminal, e todo o pré-processamento que um modelo multimodal exige, como criação dos embeddings das imagens:

In [ ]:
from deepseek_vl.utils.io import load_pil_images

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder> Analyze this image, and describe it, analysing its sentiment.",
        "images": ["/content/DeepSeek-VL/images/dog_a.png"]
    },
    {
        "role": "Assistant",
        "content": ""
    }
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

You are a helpful language and vision assistant. You are able to understand the visual content that the user provides, and assist the user with a variety of tasks using natural language.

User: <image_placeholder>Analyze this image, and classify it as one of the following sentiments: Negative, Neutral, Positive. Do not describe the image, and select only one class.

Assistant:  The image depicts a cute and playful beagle dog sitting on a blue background. The dog is wearing a black collar and has its tongue sticking out, indicating a sense of happiness or excitement. The dog's eyes are looking directly at the camera, creating a sense of connection with the viewer. The image does not contain any text or other objects. The dog is the only subject in the image, and its position on the blue background makes it the focal point of the image. The image does not provide any information about the dog's breed or any other details beyond what is visible in the image.


## Inferência em Batch

Caso dê tudo certo, vamos agora tentar um processamento em batch.

Como já foi detalhado no material, vamos focar na task 1. então focamos puramente em passar a imagem, e um prompt para o modelo.

Neste exemplo, utilizamos o subconjunto alpha5 p3, onde todos os 5 examinadores concordaram no sentimento de uma imagem, com três classes.

Mais detalhes, e acesso para essa base de dados completa podem ser acessados [aqui](https://arxiv.org/abs/2508.16873)

In [ ]:
import pandas as pd

df = pd.read_csv('https://drive.google.com/uc?export=download&id=1OLFvsOI_YPi4HxGBqjmI68uqw4dmmNT9')

df.head()

,sentiment,url,id
0,1,https://drive.google.com/uc?export=view&id=1px...,1BhIkrf3HWatQFG7KxJK7MZ1oWbUczaar
1,2,https://drive.google.com/uc?export=view&id=1Ym...,15LVGeXllPyPe1jicnPTPl6kbBYJ5lZh8
2,1,https://drive.google.com/uc?export=view&id=1Ik...,11_yZVpq48fA-jchrZO_sTtUKQejXd-te
3,1,https://drive.google.com/uc?export=view&id=1OQ...,1l-Ygs1ezj1Z68h8wFYYRb12h1_PpA8z7
4,1,https://drive.google.com/uc?export=view&id=1rf...,1sjemBBOHMg-DjQuIwbe1e_e0x7puSvWP


Depois de carregado nosso conjunto, vamos passar por cada uma das imagens, passando para o modelo a imagem, e o nosso prompt, propriamente preparado, e mostrar a reposta dele! Note que para esse modelo, extrair o sentimento exato da resposta não é uma tarefa simples, mas em modelos mais avançados, essa tarefa se torna um pouco mais fácil, já que tendem a dar uma resposta mais assertiva.

In [ ]:
import requests
import base64

for index, row in df.iterrows():
  image_url = row["url"]
  response = requests.get(image_url)
  b64response = uri = ("data:" +
                response.headers['Content-Type'] + ";" +
                "base64," + str(base64.b64encode(response.content).decode("utf-8")))

  conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>Analyze this image, and describe it, analysing its sentiment.",
        "images": [b64response]
    },
    {
        "role": "Assistant",
        "content": ""
    }
  ]

  pil_images = load_pil_images(conversation)
  prepare_inputs = vl_chat_processor(
      conversations=conversation,
      images=pil_images,
      force_batchify=True
  ).to(vl_gpt.device)

  inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

  outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
  )

  answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
  print(f"{prepare_inputs['sft_format'][0]}", answer)




You are a helpful language and vision assistant. You are able to understand the visual content that the user provides, and assist the user with a variety of tasks using natural language.

User: <image_placeholder>Analyze this image, and classify it as one of the following sentiments: Negative, Neutral, Positive. Do not describe the image, and select only one class.

Assistant:  The image depicts a group of people standing on a sidewalk, with a building in the background. The people are dressed in winter clothing, suggesting a cold weather setting. The sidewalk appears to be wet, indicating recent rainfall or snowfall. The building in the background has a sign on it, but the text is not visible in the image. The overall atmosphere of the image is somewhat gloomy, with the wet sidewalk and the people's winter clothing.
You are a helpful language and vision assistant. You are able to understand the visual content that the user provides, and assist the user with a variety of tasks using na

KeyboardInterrupt: 